In [ ]:
# __init.py file
from __future__ import print_function
from __future__ import division

In [ ]:
# bow.py, here applying tfidf + Logistic Regression and some data preprocessing
import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import loadWord2Vec
from math import log

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.svm import LinearSVC
# build corpus
dataset = '20ng' # 20 ng is dataset which one store in data/20ng directorey

# shulffing
doc_name_list = []
doc_train_list = []
doc_test_list = []

f = open('data/' + dataset + '.txt', 'r') # open 'data' directory, open the '20ng' dataset as a text extension,only read,here we find only doc_name
for line in f.readlines():  # iterate through the each line 
    doc_name_list.append(line.strip()) # each line of dataset we append it onto list,strip() method returns a copy of the 
    # string by removing both leading and trailing charachters
    temp = line.split("\t") # this line divide by each word and store it to temp list,for checking 'test' word
    if temp[1].find('test') != -1: # if the first line store the name 'test' 
        doc_test_list.append(line.strip()) # if this line is test then apend in 'doc_test_list',strip means remove leading and trailing char 
    elif temp[1].find('train') != -1: # the first line has 'train' name(checks which are not match with -1)
        doc_train_list.append(line.strip()) # if the line has train then store it to 'doc_train_list' 
f.close()  # close the python file

doc_content_list = [] 
f = open('data/corpus/' + dataset + '.clean.txt', 'r') # open the data corpus as read mode,here we find the doc_content
for line in f.readlines(): # each line of readlines file
    doc_content_list.append(line.strip()) # we append without first and last word(using strip) in a list
f.close()

train_ids = []
for train_name in doc_train_list: # 'doc_train_list' store all of the train data 
    train_id = doc_name_list.index(train_name) # index rutrns the index number of each line of 'doc_train_list'
    train_ids.append(train_id) # append the all 'train_id' in 'train_ids' list
print(train_ids)
random.shuffle(train_ids) # randomly shuffle the all train_ids

# partial labeled data


f = open('data/' + dataset + '.train.index', 'r')  # take the partial labeled data
lines = f.readlines() # take the all lines in 'lines' variable
f.close() # close the file
train_ids = [int(x.strip()) for x in lines] # each line of partial labeled data we remove first and last word and store it to 'train_ids' 


#train_ids = train_ids[:int(0.2 * len(train_ids))]

test_ids = []
for test_name in doc_test_list: # take the each line of 'doc_test_list'
    test_id = doc_name_list.index(test_name) # find the index of each 'test_name' and store it to 'test_id'
    test_ids.append(test_id)  # append the all test_id into 'test_ids' list
print(test_ids)
random.shuffle(test_ids) # now shuffle the test_ids list,here it doesn't preserve a order

ids = train_ids + test_ids # take the all train_ids and test_ids and print it
print(ids)
print(len(ids))

train_size = len(train_ids) # take the length of 'train_ids'
val_size = int(0.1 * train_size) # if the 'train_size' is 32,then val_size is 3
real_train_size = train_size - val_size # now real_train_size is 32-3 = 29

shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in ids: # here iterate through all train_ids and test_ids(which are shuffle up previously)
    shuffle_doc_name_list.append(doc_name_list[int(id)]) # store all doc_name where ids are previously shuffled(from 'data' directory)
    shuffle_doc_words_list.append(doc_content_list[int(id)]) # store all doc_content where ids are previously shuffled (from 'data/corpus' directory)

tfidf_vec = TfidfVectorizer() #max_features=50000, here 'tfidf_vec' variable defines the TfidfVectorizer()
tfidf_matrix = tfidf_vec.fit_transform(shuffle_doc_words_list) # applying tfidf in 'shuffle_doc_words_list(which contains doc_content)'
print(tfidf_matrix)
#tfidf_matrix_array = tfidf_matrix.toarray()

# BOW TFIDF + LR

#train_x = []
train_y = []

#test_x = []
test_y = []

for i in range(len(shuffle_doc_words_list)): # iterate untill len of shuffled 'doc_content'
    doc_words = shuffle_doc_words_list[i] # store each 'shuffle_doc_words' as a 'doc_words'
    words = doc_words.split(' ') # here each doc words split single words with a space 

    doc_meta = shuffle_doc_name_list[i] # store each 'shuffle_doc_name' as a 'doc_meta'
    temp = doc_meta.split('\t')  # split the doc_meta with a tab and store it to 'temp' variable
    label = temp[2] # 2nd index contains the label

    if i < train_size: # when the length is smaller then train_size then the label of train store it to 'train_y' list
        #train_x.append(tfidf_matrix_array[i])
        train_y.append(label)
    else:   # if the length is greater then train_size which means it is test data then the label append to 'test_y' lsit
        #test_x.append(tfidf_matrix_array[i])
        test_y.append(label)

#clf = svm.SVC(decision_function_shape='ovr', class_weight="balanced",kernel='linear') # if we use svm classifier then use it
#clf = LinearSVC(random_state=0) 
clf = LogisticRegression(random_state=1) # here declare LogisticRegression as a 'clf',and you have to define the random_state

clf.fit(tfidf_matrix[:train_size], train_y) # applying Log Reg untill the length of train_size,here first argument is train data with tfidf using train size,2nd argument is train data label(o/p) 
predict_y = clf.predict(tfidf_matrix[train_size:]) # here predict the o/p using test_data(where applying tfidf,using test_data size), [:train_size] means start with 0th index until 'train_data' size and [train_data:] means after the size of 'train_data' untill last means test data

correct_count = 0
for i in range(len(test_y)): # here count the predicted test o/p match with original test o/p(label) and count it
    if predict_y[i] == test_y[i]: # if the original o/p(label) is same as test o/p then increase the count 
        correct_count += 1

accuracy = correct_count * 1.0 / len(test_y) # find the test accuracy,you can omit 1,divide by len of test_data,if you try to find the per. then mult with 100
print(dataset, accuracy)

print("Precision, Recall and F1-Score...")
print(metrics.classification_report(test_y, predict_y, digits=4)) # here predict the Precisin, Recall and F1-Score,the parameter is test o/p(label),predicted o/p and takes 4 number after decimal points


In [ ]:
import re
# build corpus, Read the data corpus then write this corpus in 'data/corpus' directory as a 'corpus_str.txt' file

dataset = '20ng' # using 20ng dataset

f = open('data/' + dataset + '.txt', 'r') # '20ng' dataset where store into 'data' directory on read mode,read the 'data' directory which file has '.txt' extension
lines = f.readlines() # the readlines() method returns a list containing each line in a file as a list item
docs = []
for line in lines: # iterate each line in on all of lines where each line as a list item
    temp = line.split("\t") # each line make a list of words with space and store it to 'temp' variable,make a list of words with tab using split
    doc_file = open(temp[0], 'r') # take the 0th index word of 'temp' variable in 'doc_file'
    doc_content = doc_file.read() # read the doc_file and store it to doc_content variable
    doc_file.close() # close the file
    print(temp[0], doc_content) # print the lines of first word,and doc_content which store the read mode of doc_file
    doc_content = doc_content.replace('\n', ' ') # replace new line with a space in a 'doc_content'
    docs.append(doc_content) # each 'doc_content' append in docs list


corpus_str = '\n'.join(docs) # after each docs here add a 'new line' automatically
f.close()

f = open('data/corpus/' + dataset + '.txt', 'w') # open a file from 'data/corpus' using write purpose
f.write(corpus_str) # write this 'corpus_str' into 'data/corpus' directory
f.close()


'''
# datasets from PTE paper
f = open('data/dblp/label_train.txt', 'r')
lines = f.readlines()
f.close()

doc_id = 0
doc_name_list = []
for line in lines:
    string = str(doc_id) + '\t' + 'train' + '\t' + line.strip()
    doc_name_list.append(string)
    doc_id += 1

f = open('data/dblp/label_test.txt', 'r')
lines = f.readlines()
f.close()

for line in lines:
    string = str(doc_id) + '\t' + 'test' + '\t' + line.strip()
    doc_name_list.append(string)
    doc_id += 1

doc_list_str = '\n'.join(doc_name_list)

f = open('data/dblp.txt', 'w')
f.write(doc_list_str)
f.close()

# TREC, R8, R52, WebKB

dataset = 'R52'

f = open('data/' + dataset + '/train.txt', 'r')
lines = f.readlines()
f.close()

doc_id = 0
doc_name_list = []
doc_content_list = []

for line in lines:
    line = line.strip()
    label = line[:line.find('\t')]
    content = line[line.find('\t') + 1:]
    string = str(doc_id) + '\t' + 'train' + '\t' + label
    doc_name_list.append(string)
    doc_content_list.append(content)
    doc_id += 1

f = open('data/' + dataset + '/test.txt', 'r')
lines = f.readlines()
f.close()

for line in lines:
    line = line.strip()
    label = line[:line.find('\t')]
    content = line[line.find('\t') + 1:]
    string = str(doc_id) + '\t' + 'test' + '\t' + label
    doc_name_list.append(string)
    doc_content_list.append(content)
    doc_id += 1

doc_list_str = '\n'.join(doc_name_list)

f = open('data/' + dataset + '.txt', 'w')
f.write(doc_list_str)
f.close()

doc_name_list_str = '\n'.join(doc_name_list)

f = open('data/' + dataset + '.txt', 'w')
f.write(doc_list_str)
f.close()

doc_content_list_str = '\n'.join(doc_content_list)

f = open('data/corpus/' + dataset + '.txt', 'w')
f.write(doc_content_list_str)
f.close()
'''


In [ ]:
# build graph

import os
import random
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from utils import loadWord2Vec, clean_str
from math import log
from sklearn import svm
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
from scipy.spatial.distance import cosine

# sys.argv() is an array for command line arguments in python,len() function is used to count the number of arguments passed to 
# the command line 
if len(sys.argv) != 2: 
	sys.exit("Use: python build_graph.py <dataset>")

datasets = ['20ng', 'R8', 'R52', 'ohsumed', 'mr'] # all datasets is in the 'datasets' list

dataset = sys.argv[1] # build corpus 
 
if dataset not in datasets: 
	sys.exit("wrong dataset name")

# Read Word Vectors
# word_vector_file = 'data/glove.6B/glove.6B.300d.txt'
# word_vector_file = 'data/corpus/' + dataset + '_word_vectors.txt'
#_, embd, word_vector_map = loadWord2Vec(word_vector_file)
# word_embeddings_dim = len(embd[0])

word_embeddings_dim = 300 # declare the dim of word_embeddings 
word_vector_map = {} # started from here()

# shulffing
doc_name_list = []
doc_train_list = []
doc_test_list = []

f = open('data/' + dataset + '.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_name_list.append(line.strip())
    temp = line.split("\t")
    if temp[1].find('test') != -1:
        doc_test_list.append(line.strip())
    elif temp[1].find('train') != -1:
        doc_train_list.append(line.strip())
f.close()
# print(doc_train_list)
# print(doc_test_list)

doc_content_list = []
f = open('data/corpus/' + dataset + '.clean.txt', 'r')
lines = f.readlines()
for line in lines:
    doc_content_list.append(line.strip())
f.close()
# print(doc_content_list)

train_ids = []
for train_name in doc_train_list:
    train_id = doc_name_list.index(train_name)
    train_ids.append(train_id)
print(train_ids)
random.shuffle(train_ids)

# partial labeled data
#train_ids = train_ids[:int(0.2 * len(train_ids))]

train_ids_str = '\n'.join(str(index) for index in train_ids)
f = open('data/' + dataset + '.train.index', 'w')
f.write(train_ids_str)
f.close()

test_ids = []
for test_name in doc_test_list:
    test_id = doc_name_list.index(test_name)
    test_ids.append(test_id)
print(test_ids)
random.shuffle(test_ids)

test_ids_str = '\n'.join(str(index) for index in test_ids)
f = open('data/' + dataset + '.test.index', 'w')
f.write(test_ids_str)
f.close()

ids = train_ids + test_ids
print(ids)
print(len(ids))

shuffle_doc_name_list = []
shuffle_doc_words_list = []
for id in ids:
    shuffle_doc_name_list.append(doc_name_list[int(id)])
    shuffle_doc_words_list.append(doc_content_list[int(id)])
shuffle_doc_name_str = '\n'.join(shuffle_doc_name_list)
shuffle_doc_words_str = '\n'.join(shuffle_doc_words_list)

f = open('data/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_name_str)
f.close()

f = open('data/corpus/' + dataset + '_shuffle.txt', 'w')
f.write(shuffle_doc_words_str)
f.close()

# build vocab
word_freq = {}
word_set = set()
for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    for word in words:
        word_set.add(word)
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

vocab = list(word_set)
vocab_size = len(vocab)

word_doc_list = {}

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    appeared = set()
    for word in words:
        if word in appeared:
            continue
        if word in word_doc_list:
            doc_list = word_doc_list[word]
            doc_list.append(i)
            word_doc_list[word] = doc_list
        else:
            word_doc_list[word] = [i]
        appeared.add(word)

word_doc_freq = {}
for word, doc_list in word_doc_list.items():
    word_doc_freq[word] = len(doc_list)

word_id_map = {}
for i in range(vocab_size):
    word_id_map[vocab[i]] = i

vocab_str = '\n'.join(vocab)

f = open('data/corpus/' + dataset + '_vocab.txt', 'w')
f.write(vocab_str)
f.close()

'''
Word definitions begin
'''
'''
definitions = []

for word in vocab:
    word = word.strip()
    synsets = wn.synsets(clean_str(word))
    word_defs = []
    for synset in synsets:
        syn_def = synset.definition()
        word_defs.append(syn_def)
    word_des = ' '.join(word_defs)
    if word_des == '':
        word_des = '<PAD>'
    definitions.append(word_des)

string = '\n'.join(definitions)


f = open('data/corpus/' + dataset + '_vocab_def.txt', 'w')
f.write(string)
f.close()

tfidf_vec = TfidfVectorizer(max_features=1000)
tfidf_matrix = tfidf_vec.fit_transform(definitions)
tfidf_matrix_array = tfidf_matrix.toarray()
print(tfidf_matrix_array[0], len(tfidf_matrix_array[0]))

word_vectors = []

for i in range(len(vocab)):
    word = vocab[i]
    vector = tfidf_matrix_array[i]
    str_vector = []
    for j in range(len(vector)):
        str_vector.append(str(vector[j]))
    temp = ' '.join(str_vector)
    word_vector = word + ' ' + temp
    word_vectors.append(word_vector)

string = '\n'.join(word_vectors)

f = open('data/corpus/' + dataset + '_word_vectors.txt', 'w')
f.write(string)
f.close()

word_vector_file = 'data/corpus/' + dataset + '_word_vectors.txt'
_, embd, word_vector_map = loadWord2Vec(word_vector_file)
word_embeddings_dim = len(embd[0])
'''

'''
Word definitions end
'''

# label list
label_set = set()
for doc_meta in shuffle_doc_name_list:
    temp = doc_meta.split('\t')
    label_set.add(temp[2])
label_list = list(label_set)

label_list_str = '\n'.join(label_list)
f = open('data/corpus/' + dataset + '_labels.txt', 'w')
f.write(label_list_str)
f.close()

# x: feature vectors of training docs, no initial features
# slect 90% training set
train_size = len(train_ids)
val_size = int(0.1 * train_size)
real_train_size = train_size - val_size  # - int(0.5 * train_size)
# different training rates

real_train_doc_names = shuffle_doc_name_list[:real_train_size]
real_train_doc_names_str = '\n'.join(real_train_doc_names)

f = open('data/' + dataset + '.real_train.name', 'w')
f.write(real_train_doc_names_str)
f.close()

row_x = []
col_x = []
data_x = []
for i in range(real_train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            # print(doc_vec)
            # print(np.array(word_vector))
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_x.append(i)
        col_x.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_x.append(doc_vec[j] / doc_len)  # doc_vec[j]/ doc_len

# x = sp.csr_matrix((real_train_size, word_embeddings_dim), dtype=np.float32)
x = sp.csr_matrix((data_x, (row_x, col_x)), shape=(
    real_train_size, word_embeddings_dim))

y = []
for i in range(real_train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    y.append(one_hot)
y = np.array(y)
print(y)

# tx: feature vectors of test docs, no initial features
test_size = len(test_ids)

row_tx = []
col_tx = []
data_tx = []
for i in range(test_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i + train_size]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_tx.append(i)
        col_tx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_tx.append(doc_vec[j] / doc_len)  # doc_vec[j] / doc_len

# tx = sp.csr_matrix((test_size, word_embeddings_dim), dtype=np.float32)
tx = sp.csr_matrix((data_tx, (row_tx, col_tx)),
                   shape=(test_size, word_embeddings_dim))

ty = []
for i in range(test_size):
    doc_meta = shuffle_doc_name_list[i + train_size]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ty.append(one_hot)
ty = np.array(ty)
print(ty)

# allx: the the feature vectors of both labeled and unlabeled training instances
# (a superset of x)
# unlabeled training instances -> words

word_vectors = np.random.uniform(-0.01, 0.01,
                                 (vocab_size, word_embeddings_dim))

for i in range(len(vocab)):
    word = vocab[i]
    if word in word_vector_map:
        vector = word_vector_map[word]
        word_vectors[i] = vector

row_allx = []
col_allx = []
data_allx = []

for i in range(train_size):
    doc_vec = np.array([0.0 for k in range(word_embeddings_dim)])
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_len = len(words)
    for word in words:
        if word in word_vector_map:
            word_vector = word_vector_map[word]
            doc_vec = doc_vec + np.array(word_vector)

    for j in range(word_embeddings_dim):
        row_allx.append(int(i))
        col_allx.append(j)
        # np.random.uniform(-0.25, 0.25)
        data_allx.append(doc_vec[j] / doc_len)  # doc_vec[j]/doc_len
for i in range(vocab_size):
    for j in range(word_embeddings_dim):
        row_allx.append(int(i + train_size))
        col_allx.append(j)
        data_allx.append(word_vectors.item((i, j)))


row_allx = np.array(row_allx)
col_allx = np.array(col_allx)
data_allx = np.array(data_allx)

allx = sp.csr_matrix(
    (data_allx, (row_allx, col_allx)), shape=(train_size + vocab_size, word_embeddings_dim))

ally = []
for i in range(train_size):
    doc_meta = shuffle_doc_name_list[i]
    temp = doc_meta.split('\t')
    label = temp[2]
    one_hot = [0 for l in range(len(label_list))]
    label_index = label_list.index(label)
    one_hot[label_index] = 1
    ally.append(one_hot)

for i in range(vocab_size):
    one_hot = [0 for l in range(len(label_list))]
    ally.append(one_hot)

ally = np.array(ally)

print(x.shape, y.shape, tx.shape, ty.shape, allx.shape, ally.shape)

'''
Doc word heterogeneous graph
'''

# word co-occurence with context windows
window_size = 20
windows = []

for doc_words in shuffle_doc_words_list:
    words = doc_words.split()
    length = len(words)
    if length <= window_size:
        windows.append(words)
    else:
        # print(length, length - window_size + 1)
        for j in range(length - window_size + 1):
            window = words[j: j + window_size]
            windows.append(window)
            # print(window)


word_window_freq = {}
for window in windows:
    appeared = set()
    for i in range(len(window)):
        if window[i] in appeared:
            continue
        if window[i] in word_window_freq:
            word_window_freq[window[i]] += 1
        else:
            word_window_freq[window[i]] = 1
        appeared.add(window[i])

word_pair_count = {}
for window in windows:
    for i in range(1, len(window)):
        for j in range(0, i):
            word_i = window[i]
            word_i_id = word_id_map[word_i]
            word_j = window[j]
            word_j_id = word_id_map[word_j]
            if word_i_id == word_j_id:
                continue
            word_pair_str = str(word_i_id) + ',' + str(word_j_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1
            # two orders
            word_pair_str = str(word_j_id) + ',' + str(word_i_id)
            if word_pair_str in word_pair_count:
                word_pair_count[word_pair_str] += 1
            else:
                word_pair_count[word_pair_str] = 1

row = []
col = []
weight = []

# pmi as weights

num_window = len(windows)

for key in word_pair_count:
    temp = key.split(',')
    i = int(temp[0])
    j = int(temp[1])
    count = word_pair_count[key]
    word_freq_i = word_window_freq[vocab[i]]
    word_freq_j = word_window_freq[vocab[j]]
    pmi = log((1.0 * count / num_window) /
              (1.0 * word_freq_i * word_freq_j/(num_window * num_window)))
    if pmi <= 0:
        continue
    row.append(train_size + i)
    col.append(train_size + j)
    weight.append(pmi)

# word vector cosine similarity as weights

'''
for i in range(vocab_size):
    for j in range(vocab_size):
        if vocab[i] in word_vector_map and vocab[j] in word_vector_map:
            vector_i = np.array(word_vector_map[vocab[i]])
            vector_j = np.array(word_vector_map[vocab[j]])
            similarity = 1.0 - cosine(vector_i, vector_j)
            if similarity > 0.9:
                print(vocab[i], vocab[j], similarity)
                row.append(train_size + i)
                col.append(train_size + j)
                weight.append(similarity)
'''
# doc word frequency
doc_word_freq = {}

for doc_id in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[doc_id]
    words = doc_words.split()
    for word in words:
        word_id = word_id_map[word]
        doc_word_str = str(doc_id) + ',' + str(word_id)
        if doc_word_str in doc_word_freq:
            doc_word_freq[doc_word_str] += 1
        else:
            doc_word_freq[doc_word_str] = 1

for i in range(len(shuffle_doc_words_list)):
    doc_words = shuffle_doc_words_list[i]
    words = doc_words.split()
    doc_word_set = set()
    for word in words:
        if word in doc_word_set:
            continue
        j = word_id_map[word]
        key = str(i) + ',' + str(j)
        freq = doc_word_freq[key]
        if i < train_size:
            row.append(i)
        else:
            row.append(i + vocab_size)
        col.append(train_size + j)
        idf = log(1.0 * len(shuffle_doc_words_list) /
                  word_doc_freq[vocab[j]])
        weight.append(freq * idf)
        doc_word_set.add(word)

node_size = train_size + vocab_size + test_size
adj = sp.csr_matrix(
    (weight, (row, col)), shape=(node_size, node_size))

# dump objects
f = open("data/ind.{}.x".format(dataset), 'wb')
pkl.dump(x, f)
f.close()

f = open("data/ind.{}.y".format(dataset), 'wb')
pkl.dump(y, f)
f.close()

f = open("data/ind.{}.tx".format(dataset), 'wb')
pkl.dump(tx, f)
f.close()

f = open("data/ind.{}.ty".format(dataset), 'wb')
pkl.dump(ty, f)
f.close()

f = open("data/ind.{}.allx".format(dataset), 'wb')
pkl.dump(allx, f)
f.close()

f = open("data/ind.{}.ally".format(dataset), 'wb')
pkl.dump(ally, f)
f.close()

f = open("data/ind.{}.adj".format(dataset), 'wb')
pkl.dump(adj, f)
f.close()
